# Lecture 10. Stack and Queue

 # Stack

## Abstract Stack

In [ ]:
class AbstractCollection(object):
    """ An abstract collection implemention"""
    def __init__(self, sourceCollection = None):
        """ Sets the initial state of self, which includes the
            contents of sourceCollecvtion if it's present """
        self._size = 0
        if sourceCollection:
            for item in sourceCollection:
                self.add(item)

    def isEmpty(self):
        """ Returns True if len(self) == 0, or False otherwise"""
        return len(self)==0

    def __len__(self):
        """ returns the number of items in self """
        return self._size

    def __str__(self):
        """ Returns the string representation of self """
        return '[' + ', '.join(map(str, self)) +']'

    def __add__(self, other):
        """ Returns a new bag containing the content of both"""
        result = type(self)(self)
        for item in other:
            result.add(item)
        return result

    def __eq__(self, other):
        """ return true if self equels toher, or False otherwise """
        if self is other:
            return True
        if type(self) != type(other) or len(self)!= len(other):
            return False
        otherIter = iter(other)
        for item in self:
            if item != next(otherIter):
                return False
        return True

class AbstractStack(AbstractCollection):
    """ An abstract stack implementaton  """
    def __init__(self, sourceCollection=None):
        """ Sets the initial state of self, which includes the
            contents of sourceCollection, if it's present"""
            AbstractCollection.__init__(self, sourceCollection)

    def add(self, item):
        """ Adds item to self. """
        self.push(item)

## Array Stack

In [ ]:
class Array(object):
    """ Represents an array ADT"""
    def __init__(self, capacity, fillValue=None):
        """ Capasity is the static size of the array.
        fillValue is placed at each position. """
        self._items = list()
        for count in range(capacity):
            self._items.append(fillValue)

    def __len__(self):
        """ -> The capacity of the array """
        return len(self._items)

    def __str__(self):
        """ -> the string representation of the array """
        return str(self._items)

    def __iter__(self):
        """ Supports iteration over a view of an array"""
        return iter(self._items)

    def __getitem__(self, index):
        """ Substrict operator for access at index"""
        return self._items[index]

    def __setitem__(self, index, value):
        """ Subscript operator for replacement at index. """
        self._items[index] = value

class ArrayExpanded(object):
    """ Represents an extended array ADT"""
    def __init__(self, capacity, fillValue=None):
        """ Capasity is the static size of the array.
        fillValue is placed at each position. """
        self._items = list()
        self._logicalSize = 0

        self._capasity = capacity
        self._fillValue = fillValue

        for count in range(capacity):
            self._items.append(fillValue)

    def __len__(self):
        """ -> The capacity of the array """
        return len(self._items)

    def __str__(self):
        """ -> the string representation of the array """
        return str(self._items)

    def __iter__(self):
        """ Supports iteration over a view of an array"""
        return iter(self._items)

    def __getitem__(self, index):
        """ Substrict operator for access at index"""
        if not (0 <= index < self.size()) :
            raise IndexError("Array index out of range")
        return self._items[index]

    def __setitem__(self, index, value):
        """ Subscript operator for replacement at index. """
        if not (0 <= index < self.size()) :
            raise IndexError('Array index out of range')
        self._items[index] = value

    def size(self):
        """ Returns the number of elements in the array"""
        return self._logicalSize

    def shrink(self):
        """ Decrease the physical size of the array if necessary """
        new_size = max(self._capasity, len(self)//2)
        for _ in range(len(self) -new_size):
            self._items.pop()

    def grow(self):
        """ Increase the physical size of the array if necessary """
        for _ in range(len(self)):
            self._items.append(self._fillValue)

    def insert(self, index, new_item):
        """ Insert item at index in the array """
        if self.size() == len(self):
            self.grow()
        if index >= self.size():
            self._items[self.size()] = new_item
        else:
            index = max(index, 0)
            # shift items by one
            for i in range(self.size(), index, -1):
                self._items[i] = self._items[i-1]
            self._items[index] = new_item
        self._logicalSize += 1

    def pop(self, index):
        """ Remove and return item at intex in the attay"""
        if not (0 <= index < self.size()) :
            raise IndexError('Array index out of range')
        itemToReturn = self._items[index]
        # shift elements leftwards
        for i in range(index, self.size()-1):
            self._items[i] = self._items[i+1]
        self._items[self.size()-1] = self._fillValue
        self._logicalSize -= 1
        # to make memory usage more effective
        if self.size() <= len(self) // 4 and len(self) > self._capacity:
            self.shrink()
        return itemToReturn

    def __eq__(self, other):
        """ Check if two arrays self and other are equal """
        if self is other:
            return True
        if type(self) != type(other) or self.size() != other.size():
            return False
        for index in range(self.size()):
            if self[index] != other[index]:
                return False
        return True
        

In [ ]:
class ArrayStack(AbstractStack):
    """ An array-based stack class"""

    DEFAULT_CAPACITY = 10

    def __init__(self, sourceCollection = None):
        """Sets the initial state of self, which includes the
        contents of sourceCollection, if it's present."""
        self._items = ArrayExpanded(ArrayStack.DEFAULT_CAPACITY)
        AbstractStack.__init__(self, sourceCollection)

    def __iter__(self):
        """ Support iterration over a view of self.
        Visit items from bottom to top of the stack """
        cursor = 0
        while cursor < len(self):
            yield self._items[cursor]
            cursor += 1

    def clear(self):
        """ Makes self become empty """
        self._size = 0
        self._items = Array(ArrayStack.DEFAULT_CAPACITY)


    def peek(self):
        """ Return element from the stack's top"""
        if self.isEmpty():
            raise KeyError('The stack is empty')
        return self._items[len(self) - 1]

    def push(self, item):
        """ Push element on the top of the stack """
        if len(self) == len(self._items):
            self._items.grow()
        self._items[len(self)] = item
        self._size += 1

    def pop(self):
        """ Remove and return the element from the stack's top """
        if self.isEmpty():
            raise KeyError('The stack is empty')
        revomed_item = self._items[len(self) - 1]
        self._size -= 1
        if len(self) <= len(self._items) // 4 and \
           len(self._items) >= 2 * ArrayStack.DEFAULT_CAPACITY:
           self._items.shrink()
        return revomed_item


## Linked Stack

In [ ]:
class Node(object):
    """Represents a singly linked node."""
    def __init__(self, data, next = None):
        self.data = data
        self.next = next

class LinkedStack(AbstractStack):
    """ A link-based stack implemention"""
    def __init__(self, sourceCollection = None):
        """ Sets  the initail state of self, which includes the
        contentd of sourceCollection, if it's present"""
        self._items = None
        AbstractStack.__init__(self, sourceCollection)

    def __iter__(self):
        """ Support iteration over a view of self
        Visits items from bottom to top of the stack """
        items = list()
        def visitNodes(node):
            """ Add items to the tmp list from tail to head"""
            if node is not None:
                visitNodes(node.next)
                items.append(node.data)
        visitNodes(self._items)
        return iter(items)

    def clear(self):
        """ Makes self become empty"""
        self._size = 0
        self._items = None

    def push(self, item):
        """ Adds item to the top of the stack """
        self._items =Node(item, self._items)
        self._size += 1

    def peek(self):
        """ Return value from the stacks top """
        if self.isEmpty():
            raise KeyError('The stack is empty')
        return self._items.data

    def pop(self):
        """ Remove and return the element from the stack's top """
        if self.isEmpty():
            raise KeyError("The stack is empty")
        removed_item = self._items.data
        self._items = self._items.next
        self._size -= 1
        return removed_item


# Queue

## Abstract Queue

In [ ]:
class AbstractCollection(object):
    """ An abstract collection implementation """

    def __init__(self, sourceCollection = None):
        """ Sets the initial state of self, which includes the
        contents of sourcecollection, if it's present """
        self._size = 0
        if sourceCollection:
            for item in sourceCollection:
                self.add(item)
            

    def __len__(self):
        """ Return number of elements in self """
        return self._size

    def isEmpty(self):
        """ Check if self is empty """
        return len(self) == 0

    def __str__(self):
        """ Returns the string representation """
        return '['+', '.join(map(str, self)) +']'
    
    def __add__(self, other):
        """ Returns a new collection consisting with elements of both """
        result = type(self)(self)
        for item in other:
            result.add(item)

    def __eq__(self, other):
        """ Check if self and other are equal"""
        if self is other:
            return True
        if type(self) != type(other) or len(self) != len(other):
            return False
        otherIter = iter(other)
        for item in self:
            if item != next(otherIter):
                return False
        return True



## Array Queue

In [ ]:
class Array:
    """ Represents an array """

    def __init__(self, capacity, fillValue = None):
        """ Capasity is the static size of the array.
        Place fillValue st each position """
        self._items = list()
        for _ in range(capacity):
            self._items.append(fillValue)

    def __len__(self):
        """ return the size of the array """
        return len(self._items)
    def __str__(self):
        """ The string representation of the array"""
        return str(self._items)
    def __iter__(self):
        """ Support iteration over an array """
        return iter(self._items)

    def __getitem(self, index):
        """ Substrict operator for access at index """
        return self._items[index]

    def __setitem__(self, index, item):
        """ Substrict oparator for replacement at index """
        self._items[index] = item
        

In [ ]:
class ArrayQueue(AbstractCollection):
    """ An array-based queue implementation """

    DEFAULT_CAPACITY = 10
    
    def __init__(self, sourceCollection=None):
        """ Sets the initail state of sel, which includes the 
        contents of sourceCollection, if it's present"""
        self._front  = self._rear = -1
        self._items = Array(ArrayQueue.DEFAULT_CAPASITY)
        AbstractCollection.__init__(self, sourceCollection)

    def __iter__(self):
        """ Supports iteration over a view of self """
        cursor = self._front
        while cursor != self._rear:
            yield self._items[cursor]
            if cursor == len(self._items) - 1:
                cursor = 0
            else:
                cursor += 1
        if cursor == self._rear and cursor != -1:
            yield self._items[cursor]

    def peek(self):
        """ Return the item at the front of the queue """
        if self.isEmpty():
            raise KeyError('Queue is empty')
        return self._items[len(self) - 1]

    def clear(self):
        """ Makes self empty"""
        self._size = 0
        self._front = self._rear = -1
        self._items = Array(ArrayQueue.DEFAULT_CAPACITY)


    def add(self, item):
        """ Insert item at the rear of the queue"""
        if len(self) == len(self._items):
            temp = Array(len(self._items*2))
            # Copy data from position front through the end of the array
            i = 0
            for j in range(self._front, len(self)):
                temp[i] = self._items[j]
                i += 1
            if self._rear < len(self) - 1:
                # Copy data from position 0 through the rear
                for j in range(0, self._rear + 1)
                    temp[i] = self._items[j]
                    i += 1
            self._items = temp
            self._front = 0
            self._rear = len(self) - 1

        if self.isEmpty():
            self._rear = self._front = 0
        elif self._rear == len(self._items) - 1:
            self._rear = 0
        else:
            self._rear += 1
        
        self._items[self._rear] = item
        self._size += 1

    def remove(self, index):
        """ Remove and return item at queue
            index in range 0(head), size - 1 (reat)"""
        if not (0 <= index < len(self)):
            raise AttributeError('Index out of range')
        scaled_index = (index + self._front) % len(self._items)
        removed_item = self._items[scaled_index]
        self._size -= 1
        if self.isEmpty():
            self._rear = self._front = -1
        # scaled index before the rear
        elif scaled_index < self._rear:
            for i in range(scaled_index, self._rear):
                self._items[i] = self._items[i+1]
            self._rear -= 1
        # rear has wrapped around the array
        else:
            for i in range(scaled_index, self._front, -1):
                self._items[i] = self._items[i-1]
            self._front += 1
        return removed_item

    def pop(self):
        """ Remove and return item from at the front of the queue """
        if self.isEmpty():
            raise KeyError("Queue is empty")
        removed_item = self._items[self._front]
        self._size -= 1
        # change front marker
        if self.isEmpty():
            self._front = self._rear = -1
        elif self._front == len(self._items) - 1:
            self._front = 0
        else:
            self._front += 1
        # resize array if needed
        if len(self) <= 0.25*len(self._items) and\
            ArrayQueue.DEFAULT_CAPACITY <= len(self._items)//2:
            tempArray = Array(len(self._items)//2)
            i = 0
            for item in self:
                tempArray[i] = item
                i += 1
            self._items= tempArray
            if not self.isEmpty():
                self._fornt = 0
                self._rear = len(self) - 1
        return removed_item    

## Linked Queue

In [ ]:
class Node:
    """ Represents a singly linked Node """
    def __init__(self, data, next=None):
        self.data = data
        self.next = next

In [ ]:
class LinkedArray(AbstractCollection):
    """ A link based queue implemention """

    def __init__(self, sourceCollection = None):
        """ Sets the initial state of self, which indicates the 
        contents of souercecollection, if it's present """
        self._front = self._rear = None
        AbstractCollection.__init(self, sourceCollection)

    def __iter__(self):
        """ Supports iteration over a view of self """
        cursor = self._head
        while cursor is not None:
            yield cursor.data
            cursor = cursor.next

    def peek(self):
        """ Returns the item at the front of the queue
        Precondition: the queue is not empty
        Raises: KeyError if stack is empty"""
        if self.isEmpty():
            raise KeyError('The queue is empty')
        return self._front.data

    def clear(self):
        """ Makes self become empty """
        self._size = 0
        self._front = self._rear = None

    def add(self, item):
        """ Adds item to the rear of the queue """
        new_node = Node(item, None)
        if self.isEmpty():
            self._front = new_node
        else:
            self._rear.next = new_node
        self._rear = new_node
        self._size += 1

    def pop(self):
        """ Remove and return the item in the front of the queue 
        Predictions: queue is not empty
        Raises: KeyError if queue is empty
        Postcondiction: the front element is removed from the queue"""
        if self.isEmpty():
            raise KeyError('Index out of range')
        removed_item =self._front.data
        self._front = self._front.next
        if self._front is None:
            self._rear = None
        self._size -= 1
        return removed_item            


    def remove(self, index):
        """ Remove and return the item by it's index
        Predictions: index is from 0 (front) to the (size - 1) """
        if not (0 <= index < len(self)):
            raise AttributeError('Index out of range')
        if index == 0:
            removed_item = self._front.data
            self._front = self._front.next
        else:
            current = self._front
            while index > 1:
                index -= 1
                current = current.next
            removed_item = current.next.data
            current.next = current.next.next
        self._size -= 1
        if self.isEmpty():
            self._rear = None
        return removed_item

# Deque

## Abstract Deque

In [ ]:
class AbstractCollection(object):
    """ An abstract collection implementation """

    def __init__(self, sourceCollection):
        """ Sets the initial state of self, which indicates the
        contents of sourceCollection, if it's present"""
        self._size = 0
        if sourceCollection:
            for item in sourceCollection:
                self.add(item)

    def __str__(self):
        """ string representation of the of self"""
        return '['   +  ', '.join(map(str, self)) +  ']'

    def isEmpty(self):
        """ chack if self is empty"""
        return self._size == 0

    def __len__(self):
        """ Return number of elements in self"""
        return self._size

    def __add__(self, other):
        """ returrn the new collection that contains both items """
        result = type(self)(self)
        for item in other:
            result.add(item)
        return result

    def __eq__(self, other):
        """ Check if self equels other """
        if self is other:
            return True
        if type(self) != type(other) or len(self) != len(other):
            return False
        otherIter = iter(other)
        for item in self:
            if item!= next(otherIter):
                return False
        return True

## Linked Deque

In [ ]:
class Node:
    """ Node class with two way links"""

    def __init__(self, data, previous=None, next=None):
        """ Set the initial state with next and previous default as None"""
        self.data = data
        self.next = next
        self.previous = previous


In [ ]:
class LinkedDeque(AbstractCollection):
    """ A link-based deque implementation"""

    def __init__(self, sourceCollection=None):
        """ Sets the initial state of self, which includes the
        contents of sourceCollecion, if it's present """
        self._front = self._rear = None
        AbstractCollection.__init__(self, sourceCollection)

    def __iter__(self):
        """Supports iteration over a view of self."""
        cursor = self._front
        while not cursor is None:
            yield cursor.data
            cursor = cursor.next

    def assert_index_range(self, index):
        """ Check if index is in range from 0 to len(self) -1.
        On occasion of wrong index raise IndexError"""
        if not (-1< index < len(self)):
            assert IndexError('Index out of range')

    def assert_emptiness(self):
        """ Check if self is empty, raise KeyError, on that case"""
        if self.isEmpty():
            raise KeyError('The deque is empty')

    def first(self):
        """ Return item at the deque front """
        self.assert_emptiness()
        return self._front.data

    def last(self):
        """ Return item at the deque rear """
        self.assert_emptiness()
        return self._rear.data

    def clear(self):
        """ Make self empty"""
        self._front = self._rear = None
        self._size = 0

    def __getitem__(self, index):
        """ substitution operator to access item at index """
        self.assert_emptiness()
        self.assert_index_range(index)
        current =  self._front
        for _ in range(index):
            current = current.next
        return current.data

    def __setitem__(self, index, item):
        """ substitution operator to chancge item at index """
        self.assert_emptiness()
        self.assert_index_range(index)
        current =  self._front
        for _ in range(index):
            current = current.next
        current.data = item

    def pop(self):
        """ Remove and return item from the deque's end"""
        self.assert_emptiness()
        removed_item = self._front
        self._front = self._front.next
        self._size -= 1
        if self._front is None:
            self._rear = None
        return removed_item

    def append(self, item):
        """ Add item to the deque'd end"""
        new_node = Node(item, previous=self._rear)
        if self.isEmpty():
            self._front = new_node
        else:
            self._rear.next = new_node
        self._rear = new_node
        self._size += 1


    def popleft(self):
        """ Remove and return item from the deque's end"""
        self.assert_emptiness()
        removed_item = self._rear
        self._rear = self._rear.previous
        self._rear.next = None
        self._size -= 1
        if self._rear is None:
            self._front = None
        return removed_item

    def appendleft(self, item):
        """ Add item to the deque's beginning"""
        new_node = Node(item, next=self._front)
        if self.isEmpty():
            self._rear = new_node
        else:
            self._front.previous = new_node
        self._front = new_node
        self._size += 1

    def remove(self, index):
        """ Remove and return item from the deque by its index"""
        if index == 0:
            return self.pop()
        if index in [-1, len(self)-1]:
            return self.popleft()
        self.assert_emptiness()
        self.assert_index_range(index)
        current = self._front
        while index:
            if current is None:
                raise ValueError('Item not in deque')
            current = current.next
            index -= 1
        current.next.previous = current.previous
        current.previous.next = current.next
        return current.data


    def index(self, item):
        """ Find and return index of the item in deque, if is present, else -1"""
        current = self._front
        index = 0
        while current is not None and current.data != item:
            current = current.next
            index += 1
        return index if current is not None else -1

    def remove_item(self, item):
        """ remove and return the item by it's index from the deque's front"""
        self.assert_emptiness()
        current = self._front
        while current is not None and current.data != item:
            current = current.next
        if current is None:
            raise ValueError('Item not in deque')
        if current is self._front:
            return self.pop()
        if current is self._rear:
            return self.popleft()
        current.next.previous = current.previous
        current.previous.next = current.next
        return current.data

    def rotate(self, shift_index):
        """ Shift deque items on k rightwards"""
        self.assert_emptiness()
        shift_index = shift_index % len(self)
        current = self._rear
        if shift_index == 0:
            return
        while shift_index != 0:
            current = current.previous
            shift_index -= 1
        # connect deque ends
        self._rear.next = self._head
        self._front.previous = self._rear
        # change front and rear nodes
        self._rear = current
        self._front = current.next
        self._rear.next = self._front.previous = None

